In [ ]:
#Реализация модели частичного обучения на основе Co-Training метода
#Добавлена проверка улучшения качества моделирования на итерации
import re
import math
import time
import random
import sys
import copy

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mplib
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import sklearn as sk
from sys import platform, version
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import ensemble
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
import catboost as ctb

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn import preprocessing
scalar = preprocessing.StandardScaler()


In [ ]:
#Выбор ядра моделирования для co-training
def classifier_mode(clf_name, n_estimators=100, clf_mix_type=0 ):
    if clf_name =='CATB':
        clf1 = CatBoostClassifier(iterations=10, learning_rate=0.1, depth=2, loss_function='Logloss')
    
    elif clf_name=='XGB':
        clf1 = XGBClassifier()

    elif   clf_name == 'CLF_NAIVE_BAYES':
         clf1 = BernoulliNB()

    elif clf_name == 'CLF_LOGREG':
         clf1 = LogisticRegression(solver='newton-cg', class_weight='balanced',penalty='l2')

    elif   clf_name == 'CLF_RANDOM_FOREST':
         clf1 = ensemble.RandomForestClassifier(random_state = 0,n_estimators=n_estimators)

    elif clf_name == 'CLF_GBOOST':
        clf1 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=0.01)

    elif clf_name == 'CLF_SVM':
         clf1 = LinearSVC()
    elif clf_name == 'CLF_KNN':
         clf1 = KNeighborsClassifier(n_neighbors=n_estimators)

    elif clf_name == 'CLF_MIX':
        print('CLF_MIX')
        if clf_mix_type ==1:
            clf1 = BernoulliNB()
            clf2 = ensemble.RandomForestClassifier(random_state=10, n_estimators=n_estimators)
        elif clf_mix_type == 2:
            clf1 = BernoulliNB()
            clf2 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=10, learning_rate=0.01)
        elif clf_mix_type == 3:
            clf1 = BernoulliNB()
            clf2 = LogisticRegression(solver='newton-cg', class_weight='balanced')
        elif clf_mix_type == 4:
            clf1 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=10, learning_rate=0.01)
            clf2 = ensemble.RandomForestClassifier(random_state=10, n_estimators=n_estimators)
        elif clf_mix_type == 5:
            clf1 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=10, learning_rate=0.01)
            clf2 = LogisticRegression(solver='newton-cg', class_weight='balanced')
        elif clf_mix_type == 6:
            clf1 = LogisticRegression(solver='newton-cg', class_weight='balanced')
            clf2 = ensemble.RandomForestClassifier(random_state=10, n_estimators=n_estimators)
        elif clf_mix_type == 7:
            clf1 = LinearSVC()
            clf2 = BernoulliNB()
        elif clf_mix_type == 8:
            clf1 = LinearSVC()
            clf2 = LogisticRegression(solver='newton-cg', class_weight='balanced')
        elif clf_mix_type == 9:
            clf1 = LinearSVC()
            clf2 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=10, learning_rate=0.01)
        elif clf_mix_type == 10:
            clf1 = LinearSVC()
            clf2 = ensemble.RandomForestClassifier(random_state=10, n_estimators=n_estimators)
        elif clf_mix_type == 11:
            clf1 = KNeighborsClassifier(n_neighbors=n_estimators)
            clf2 = BernoulliNB()
        elif clf_mix_type == 12:
            clf1 = KNeighborsClassifier(n_neighbors=n_estimators)
            clf2 = LogisticRegression(solver='newton-cg', class_weight='balanced')
        elif clf_mix_type == 13:
            clf1 = KNeighborsClassifier(n_neighbors=n_estimators)
            clf2 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=10, learning_rate=0.01)
        elif clf_mix_type == 14:
            clf1 = KNeighborsClassifier(n_neighbors=n_estimators)
            clf2 = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, random_state=10, learning_rate=0.01)
        elif clf_mix_type ==15:
            clf1 = LogisticRegression(solver='newton-cg', class_weight='balanced')
            clf2 = XGBClassifier()
        elif clf_mix_type ==16:
            clf1 = LogisticRegression(solver='newton-cg', class_weight='balanced')
            clf2 = CatBoostClassifier()
        elif clf_mix_type ==17:
            clf1 = XGBClassifier()
            clf2 = CatBoostClassifier()
            

    if clf_name == 'CLF_MIX':
        return clf_name, clf1, clf2

    else:
        return clf_name, clf1


In [ ]:
#Тестирование качества на итерации метода
def CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_true = None, metric='f1_score'):
    
    if y_true is None:
        # предобработка
        X1_test_ = dataframe_test[X1]
        X2_test_ = dataframe_test[X2]
        X1_test = np.asarray(X1_test_).reshape(len(X1_test_), len(X1))
        X2_test = np.asarray(X2_test_).reshape(len(X2_test_), len(X2))

        proba_clf1_test = clf1.predict_proba(X1_test)
        proba_clf2_test = clf2.predict_proba(X2_test)
        aggr_proba_test = (proba_clf1_test + proba_clf2_test)/2

        clss_test = []
        for i in range(0, len(aggr_proba_test)):

            # максимум вероятности из произведения предсказанных вероятностей
            if aggr_proba_test[i][1] > aggr_proba_test[i][0]:
                clss_test.append(1)
            else:
                clss_test.append(0)

        return clss_test  #вернуть только предсказанные значения модели

    elif y_true is not None:
        # предобработка
        X1_test_ = dataframe_test[X1]
        X2_test_ = dataframe_test[X2]
        X1_test = np.asarray(X1_test_).reshape(len(X1_test_), len(X1))
        X2_test = np.asarray(X2_test_).reshape(len(X2_test_), len(X2))

        proba_clf1_test = clf1.predict_proba(X1_test)
        proba_clf2_test = clf2.predict_proba(X2_test)
        aggr_proba_test = proba_clf1_test * proba_clf2_test

        clss_test = []
        for i in range(0, len(aggr_proba_test)):

            # максимум вероятности из произведения предсказанных вероятностей
            if aggr_proba_test[i][1] > aggr_proba_test[i][0]:
                clss_test.append(1)
            else:
                clss_test.append(0)
        if metric=='f1_score':
            f_measure_it = sk.metrics.f1_score(y_true, clss_test, average='weighted') 
        elif metric=='precision_score':
            f_measure_it = sk.metrics.precision_score(y_true, clss_test, average='weighted') 
        elif metric=='recall_score':
            f_measure_it = sk.metrics.recall_score(y_true, clss_test, average='weighted') 
        else:
            print('Error: metric is not set')

        return f_measure_it  #вернуть качество модели

In [ ]:
#Сам метод Co-Training
def CoTrain_Classifier(dataframe, X1, X2, y,
                       clf_1, clf_2=None,
                       method ='convergency',
                       conv_mark = 'aggr_proba_max',
                       th_step = 0.001,
                       co_train_k=5,
                       aggr_proba_th = 0.3,
                       p_const = 100,
                       n = None, p = None,
                       u_marker=999,
                       objs_keys=[],
                       dataframe_test=None,
                       y_data_test = None):

    #////////////////////////////////ПАРАМЕТРЫ ЧАСТИЧНОГО ОБУЧЕНИЯ CO-TRAINING/////////////////////////////////////////
    #Для работы алгоритма необходимы библиотеки: pandas, numpy, sklearn
    #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    #dataframe (pandas) - таблица данных для обучения
    #X1 (list), str - список полей для 1 обучения
    #X2 (list), str - список полей для 2 обучения
    #y (pandas) - вектор разметки объектов
    #clf_1, clf_2 - классификаторы в модели, по умолчанию clf_2 = clf_1
    #method: подметод классификации:
        #>'k_iters' - число итераций задано
            #>co_train_k - заданное число итераций
        #>'convergency' - до полной сходимости
            #>conv_mark: разметка для конечной сходимости, если method = 'convergency'
                #>aggr_proba_max - единоразовая разметка с максимумо возможной кооперативной вероятности
                #>proba_th_min - с постепенной минимизацией критерия уверенной классификации
                    #>th_step - шаг снижения вероятности при несходимости на итерации, по умолчанию 0.001
    #aggr_proba_th - уровень уверенной кооперативной классификации объектов
    #n, p -число уверенных разметок за итерацию
    #u_marker - псевдоразметка объектов в y для классификации, по умолчанию 999
    #objs_keys (pandas) - привязка к дополнительным номинальным меткам объектов для БД, по умолчанию None
    #dataframe_test, y_true - таблица тестовых данных и соответствующий вектор классов, по умолчанию отключено


    #Задание классификаторов для обучения
    clf1 = clf_1
    if clf_2 is None:

        clf2 = copy.copy(clf1)
    else:
        clf2 = clf_2
    #Поиск индексов размеченных и неразмеченных объектов, по умолчанию u_marker=999 в y
    U = [i for i, y_i in enumerate(y) if y_i == u_marker]
    L = [i for i, y_i in enumerate(y) if y_i != u_marker]

    y_tmp_ = y.iloc[L].copy()

    df_X1_L_tmp_ = dataframe[X1].iloc[L].copy()
    df_X2_L_tmp_ = dataframe[X2].iloc[L].copy()

    df_X1_U_tmp_ = dataframe[X1].iloc[U].copy()
    df_X2_U_tmp_ = dataframe[X2].iloc[U].copy()

    y_tmp = np.asarray(y_tmp_).reshape(len(y_tmp_), 1)
    y_tmp_prob = copy.copy(y_tmp)
    
    df_X1_L_tmp = np.asarray(df_X1_L_tmp_).reshape(len(df_X1_L_tmp_), len(X1))
    df_X2_L_tmp = np.asarray(df_X2_L_tmp_).reshape(len(df_X2_L_tmp_), len(X2))

    df_X1_U_tmp = np.asarray(df_X1_U_tmp_).reshape(len(df_X1_U_tmp_), len(X1))
    df_X2_U_tmp = np.asarray(df_X2_U_tmp_).reshape(len(df_X1_U_tmp_), len(X2))

    #привязка к дополнительным номинальным меткам объектов для БД, по умолчанию None
    if len(objs_keys) == 0:
        print('objs_keys = None, Ключи для БД не заданы')

    else:
        print('objs_keys, Ключи для БД заданы')
        objs_keys_L_tmp_ = objs_keys.iloc[L].copy()
        objs_keys_U_tmp_ = objs_keys.iloc[U].copy()

        objs_keys_L_tmp = np.asarray(objs_keys_L_tmp_).reshape(len(objs_keys_L_tmp_), 1)
        objs_keys_U_tmp = np.asarray(objs_keys_U_tmp_).reshape(len(objs_keys_U_tmp_), 1)

    if method =='convergency':
        print('method = ', method)
        cotrain_iters = 0
        print(40*'/')
        print(40*'/')
        unlabeled_rows = len(df_X1_U_tmp) #общее число неразмеченных строк
        unlabeled_rows_hist = [unlabeled_rows+1]        #для проверки сходимости алгоритма

        while unlabeled_rows > 0:
            cotrain_iters += 1
            unlabeled_rows_hist.append(unlabeled_rows)
            conv_indicator = unlabeled_rows_hist[cotrain_iters-1] -  unlabeled_rows_hist[cotrain_iters]
            print('conv indi = ', conv_indicator)
            print('aggr_proba_th = ', aggr_proba_th)

            if conv_indicator > 0 :

                print(40 * '/','\n',40 * '/')
                print('Co-train iters = ', cotrain_iters)
                print(40 * '/','\n',40 * '/')

                
                
                # Обучение
                clf1.fit(df_X1_L_tmp, y_tmp.ravel())
                clf2.fit(df_X2_L_tmp, y_tmp.ravel())

                model_qual_t0 = CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_data_test)

                # Оценка вероятностей классов
                proba_clf1 = clf1.predict_proba(df_X1_U_tmp)
                proba_clf2 = clf2.predict_proba(df_X2_U_tmp)
                unmkd_tmp = len(df_X1_U_tmp)

                aggr_proba = (proba_clf1 + proba_clf2)/2  

                pos_aggr_prob_ind = np.asarray(np.where(aggr_proba[ :, 1] > aggr_proba_th)).ravel()
                neg_aggr_prob_ind = np.asarray(np.where(aggr_proba[ :, 0] > aggr_proba_th)).ravel()

                randomize_p = np.arange(len(pos_aggr_prob_ind))
                randomize_n = np.arange(len(neg_aggr_prob_ind))

                np.random.shuffle(randomize_p)
                np.random.shuffle(randomize_n)
                pos_aggr_prob_ind = pos_aggr_prob_ind[randomize_p]
                neg_aggr_prob_ind = neg_aggr_prob_ind[randomize_n]

                neg_aggr_prob_ind= neg_aggr_prob_ind[:len(pos_aggr_prob_ind)]


                conf_examples = np.concatenate((pos_aggr_prob_ind,  neg_aggr_prob_ind))

                y_clss = []        # размеченный класс
                y_clss_proba = []  # вероятность принадлежности к 1 классу


                unmkd = unmkd_tmp - len(conf_examples)
                print('Разметка данных')
                print('Неразмеченных объектов= ', unmkd)

                # добавление переменных по меткам классов
                df_X1_L_tmp_ch = copy.copy(np.vstack([df_X1_L_tmp, df_X1_U_tmp[conf_examples]]))
                df_X2_L_tmp_ch = copy.copy(np.vstack([df_X2_L_tmp, df_X2_U_tmp[conf_examples]]))

                y_clss = np.asarray(y_clss)
                y_clss = y_clss.reshape(len(y_clss), 1)
                y_clss_proba = np.asarray(y_clss_proba)
                y_clss_proba = y_clss_proba.reshape(len(y_clss_proba), 1)

                y_clss_p = np.ones( shape =(len(pos_aggr_prob_ind),1), dtype = int)
                y_clss_n = np.zeros( shape =(len(neg_aggr_prob_ind),1), dtype = int)

                #итоговые массивы для добавления
                y_clss = np.vstack([y_clss_p, y_clss_n]) 
                y_clss_proba_norm = aggr_proba[conf_examples, 1]/(aggr_proba[conf_examples, 1]+aggr_proba[conf_examples, 0])  
                y_clss_proba = np.vstack((y_clss_proba, y_clss_proba_norm.reshape(len(y_clss_proba_norm),1)))

                #приведение предыдущих массивов к единому виду
                y_tmp = np.asarray(y_tmp)
                y_tmp = y_tmp.reshape(len(y_tmp), 1)

                y_tmp_prob = np.asarray(y_tmp_prob)
                y_tmp_prob = y_tmp_prob.reshape(len(y_tmp_prob), 1)

                # добавление меток класса
                y_tmp_ch = copy.copy(np.vstack([y_tmp, y_clss]))
                y_tmp_prob_ch = copy.copy(np.vstack([y_tmp_prob, y_clss_proba]))

                # корректировка неразмеченных массивов
                # backup
                df_X1_U_tmp_backup = df_X1_U_tmp.copy()
                df_X2_U_tmp_backup = df_X2_U_tmp.copy()

                df_X1_U_tmp_ch = np.delete(df_X1_U_tmp, conf_examples, axis=0)
                df_X2_U_tmp_ch = np.delete(df_X2_U_tmp, conf_examples, axis=0)

                #Автокоррекция подгонки модели
                clf1.fit(df_X1_L_tmp_ch, y_tmp_ch.ravel())
                clf2.fit(df_X2_L_tmp_ch, y_tmp_ch.ravel())

                model_qual_t1 = CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_data_test)
                print('Качество до текущей разметки:', '{0:.5f}'.format(model_qual_t0))
                print('Качество после текущей разметки:', '{0:.5f}'.format(model_qual_t1))
                #print(len(df_X2_U_tmp_ch))
                #print(len(df_X1_U_tmp_backup))

                print('Согласованных объектов для разметки: ',len(conf_examples))
                print('Positive: ', pos_aggr_prob_ind)
                print('Negative: ', neg_aggr_prob_ind)
                print(conf_examples)
                if model_qual_t1 > model_qual_t0:
                    GREEN_LIGHT = 1

                    #k+=1
                    print('Качество повышено на ',model_qual_t1 - model_qual_t0, ', объекты размечаются')              
                    #можно объявить массивы основными данными для подгонки                
                    y_tmp = y_tmp_ch
                    y_tmp_prob = y_tmp_prob_ch

                    df_X1_L_tmp = df_X1_L_tmp_ch
                    df_X2_L_tmp = df_X2_L_tmp_ch

                    df_X1_U_tmp = df_X1_U_tmp_ch
                    df_X2_U_tmp = df_X2_U_tmp_ch

                else:
                    #k+=1 # убрать
                    GREEN_LIGHT = 0

                    df_X1_U_tmp = df_X1_U_tmp_backup
                    df_X2_U_tmp = df_X2_U_tmp_backup
                    objs_keys_U_tmp

                    randomize = np.arange(len(df_X1_U_tmp))
                    np.random.shuffle(randomize)

                    df_X1_U_tmp = df_X1_U_tmp[randomize]
                    df_X2_U_tmp = df_X2_U_tmp[randomize]
                    objs_keys_U_tmp = objs_keys_U_tmp[randomize]

                    print('Качество снижено, объекты не размечаются')


                unlabeled_rows = len(df_X1_U_tmp)


                if len(objs_keys) == 0:
                    print('objs_keys is None, Ключи для БД не заданы')
                else:
                    print('objs_keys is not None, Добавление ключей БД')
                    if GREEN_LIGHT == 1:
                        objs_keys_L_tmp = np.vstack([objs_keys_L_tmp, objs_keys_U_tmp[conf_examples]])
                        objs_keys_U_tmp = np.delete(objs_keys_U_tmp, conf_examples, axis=0)
                    else:
                        print('Добавление ключей БД отменено, качество модели не повышено')

                print('Неразмечено объектов = ', unlabeled_rows)
                print('Размечено объектов = ', len(df_X1_L_tmp))
                #print('Меток поставлено = ', len(y_tmp))
                
                
                


            elif conv_indicator ==0:
                
                if conv_mark == 'aggr_proba_max':
                    print('Корректировка сходимости, метод:',conv_mark )
                    #1 - единоразовая разметка
                    proba_clf1 = clf1.predict_proba(df_X1_U_tmp)
                    proba_clf2 = clf2.predict_proba(df_X2_U_tmp)
                    aggr_proba = (proba_clf1 + proba_clf2)/2

                    y_conv_1_proba = np.asarray([])
                    y_conv_1_proba = y_conv_1_proba.reshape(len(y_conv_1_proba), 1)

                    logic = aggr_proba[:,1]> aggr_proba[:,0] #True - класс 1, False - класс 0

                    pos = np.where(logic==True) 
                    neg = np.where(logic==False) 
                    all_ind = np.concatenate((pos[0],  neg[0]))

                    y_clss_p = np.ones( shape =(len(pos[0]),1), dtype = int)
                    y_clss_n = np.zeros( shape =(len(neg[0]),1), dtype = int)

                    y_conv_1 = np.vstack([y_clss_p, y_clss_n]) 
                    y_conv_proba_norm = aggr_proba[all_ind, 1]/(aggr_proba[all_ind, 1]+ aggr_proba[all_ind, 0])
                    y_conv_1_proba = np.vstack((y_conv_1_proba, y_conv_proba_norm.reshape(len(y_conv_proba_norm),1)))


                    # добавление переменных по меткам классов
                    df_X1_L_tmp = np.vstack([df_X1_L_tmp, df_X1_U_tmp[all_ind]])
                    df_X2_L_tmp = np.vstack([df_X2_L_tmp, df_X2_U_tmp[all_ind]])


                    y_clss = np.asarray(y_conv_1)
                    y_clss = y_clss.reshape(len(y_clss), 1)

                    y_clss_proba = np.asarray(y_conv_1_proba)
                    y_clss_proba = y_clss_proba.reshape(len(y_clss_proba), 1)

                    y_tmp = np.asarray(y_tmp)
                    y_tmp = y_tmp.reshape(len(y_tmp), 1)

                    y_tmp_prob = np.asarray(y_tmp_prob)
                    y_tmp_prob = y_tmp_prob.reshape(len(y_tmp_prob), 1)

                    y_tmp = np.vstack([y_tmp, y_clss])
                    y_tmp_prob = np.vstack([y_tmp_prob, y_clss_proba])

                    df_X1_U_tmp = np.delete(df_X1_U_tmp, all_ind, axis=0)
                    df_X2_U_tmp = np.delete(df_X2_U_tmp, all_ind, axis=0)

                    unlabeled_rows = len(df_X1_U_tmp)
                    
                    if len(objs_keys) == 0:
                        print('objs_keys is None, Ключи для БД не заданы')
                    else:
                        print('objs_keys is not None, Добавление ключей БД')
                        objs_keys_L_tmp = np.vstack([objs_keys_L_tmp, objs_keys_U_tmp])



                    print('Неразмечено объектов = ', unlabeled_rows)
                    #print('Размечено объектов = ', len(df_X1_L_tmp))
                    print('Меток поставлено = ', len(y_tmp))

                #2 - снижения границы aggr_proba_th

                elif conv_mark =='proba_th_min':
                    print('Корректировка сходимости, заданная вероятность снижена на: ',th_step )
                    aggr_proba_th = aggr_proba_th - th_step #смягчаем условие

                    print(40 * '/', '\n', 40 * '/')
                    print('Co-train iters = ', cotrain_iters)
                    print(40 * '/', '\n', 40 * '/')                   
                    
                    # Обучение
                    clf1.fit(df_X1_L_tmp, y_tmp.ravel())
                    clf2.fit(df_X2_L_tmp, y_tmp.ravel())

                    model_qual_t0 = CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_data_test)

                    # Оценка вероятностей классов
                    proba_clf1 = clf1.predict_proba(df_X1_U_tmp)
                    proba_clf2 = clf2.predict_proba(df_X2_U_tmp)
                    unmkd_tmp = len(df_X1_U_tmp)

                    aggr_proba = (proba_clf1 + proba_clf2)/2  

                    pos_aggr_prob_ind = np.asarray(np.where(aggr_proba[ :, 1] > aggr_proba_th)).ravel()
                    neg_aggr_prob_ind = np.asarray(np.where(aggr_proba[ :, 0] > aggr_proba_th)).ravel()

                    randomize_p = np.arange(len(pos_aggr_prob_ind))
                    randomize_n = np.arange(len(neg_aggr_prob_ind))

                    np.random.shuffle(randomize_p)
                    np.random.shuffle(randomize_n)
                    pos_aggr_prob_ind = pos_aggr_prob_ind[randomize_p]
                    neg_aggr_prob_ind = neg_aggr_prob_ind[randomize_n]

                    neg_aggr_prob_ind= neg_aggr_prob_ind[:len(pos_aggr_prob_ind)]


                    conf_examples = np.concatenate((pos_aggr_prob_ind,  neg_aggr_prob_ind))

                    y_clss = []        # размеченный класс
                    y_clss_proba = []  # вероятность принадлежности к 1 классу


                    unmkd = unmkd_tmp - len(conf_examples)
                    print('Разметка данных')
                    print('Неразмеченных объектов= ', unmkd)

                    # добавление переменных по меткам классов
                    df_X1_L_tmp_ch = copy.copy(np.vstack([df_X1_L_tmp, df_X1_U_tmp[conf_examples]]))
                    df_X2_L_tmp_ch = copy.copy(np.vstack([df_X2_L_tmp, df_X2_U_tmp[conf_examples]]))

                    y_clss = np.asarray(y_clss)
                    y_clss = y_clss.reshape(len(y_clss), 1)
                    y_clss_proba = np.asarray(y_clss_proba)
                    y_clss_proba = y_clss_proba.reshape(len(y_clss_proba), 1)

                    y_clss_p = np.ones( shape =(len(pos_aggr_prob_ind),1), dtype = int)
                    y_clss_n = np.zeros( shape =(len(neg_aggr_prob_ind),1), dtype = int)

                    #итоговые массивы для добавления
                    y_clss = np.vstack([y_clss_p, y_clss_n]) 
                    y_clss_proba_norm = aggr_proba[conf_examples, 1]/(aggr_proba[conf_examples, 1]+aggr_proba[conf_examples, 0])  
                    y_clss_proba = np.vstack((y_clss_proba, y_clss_proba_norm.reshape(len(y_clss_proba_norm),1)))

                    #приведение предыдущих массивов к единому виду
                    y_tmp = np.asarray(y_tmp)
                    y_tmp = y_tmp.reshape(len(y_tmp), 1)

                    y_tmp_prob = np.asarray(y_tmp_prob)
                    y_tmp_prob = y_tmp_prob.reshape(len(y_tmp_prob), 1)

                    # добавление меток класса
                    y_tmp_ch = copy.copy(np.vstack([y_tmp, y_clss]))
                    y_tmp_prob_ch = copy.copy(np.vstack([y_tmp_prob, y_clss_proba]))

                    # корректировка неразмеченных массивов
                    # backup
                    df_X1_U_tmp_backup = df_X1_U_tmp.copy()
                    df_X2_U_tmp_backup = df_X2_U_tmp.copy()

                    df_X1_U_tmp_ch = np.delete(df_X1_U_tmp, conf_examples, axis=0)
                    df_X2_U_tmp_ch = np.delete(df_X2_U_tmp, conf_examples, axis=0)

                    #Автокоррекция подгонки модели
                    clf1.fit(df_X1_L_tmp_ch, y_tmp_ch.ravel())
                    clf2.fit(df_X2_L_tmp_ch, y_tmp_ch.ravel())

                    model_qual_t1 = CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_data_test)
                    print('Качество до текущей разметки:', '{0:.5f}'.format(model_qual_t0))
                    print('Качество после текущей разметки:', '{0:.5f}'.format(model_qual_t1))
                    #print(len(df_X2_U_tmp_ch))
                    #print(len(df_X1_U_tmp_backup))

                    print('Согласованных объектов для разметки: ',len(conf_examples))
                    print('Positive: ', pos_aggr_prob_ind)
                    print('Negative: ', neg_aggr_prob_ind)
                    print(conf_examples)
                    if model_qual_t1 > model_qual_t0:
                        GREEN_LIGHT = 1

                        #k+=1
                        print('Качество повышено на ',model_qual_t1 - model_qual_t0, ', объекты размечаются')              
                        #можно объявить массивы основными данными для подгонки                
                        y_tmp = y_tmp_ch
                        y_tmp_prob = y_tmp_prob_ch

                        df_X1_L_tmp = df_X1_L_tmp_ch
                        df_X2_L_tmp = df_X2_L_tmp_ch

                        df_X1_U_tmp = df_X1_U_tmp_ch
                        df_X2_U_tmp = df_X2_U_tmp_ch

                    else:
                        #k+=1 # убрать
                        GREEN_LIGHT = 0

                        df_X1_U_tmp = df_X1_U_tmp_backup
                        df_X2_U_tmp = df_X2_U_tmp_backup
                        objs_keys_U_tmp

                        randomize = np.arange(len(df_X1_U_tmp))
                        np.random.shuffle(randomize)

                        df_X1_U_tmp = df_X1_U_tmp[randomize]
                        df_X2_U_tmp = df_X2_U_tmp[randomize]
                        objs_keys_U_tmp = objs_keys_U_tmp[randomize]

                        print('Качество снижено, объекты не размечаются')


                    unlabeled_rows = len(df_X1_U_tmp)


                    if len(objs_keys) == 0:
                        print('objs_keys is None, Ключи для БД не заданы')
                    else:
                        print('objs_keys is not None, Добавление ключей БД')
                        if GREEN_LIGHT == 1:
                            objs_keys_L_tmp = np.vstack([objs_keys_L_tmp, objs_keys_U_tmp[conf_examples]])
                            objs_keys_U_tmp = np.delete(objs_keys_U_tmp, conf_examples, axis=0)
                        else:
                            print('Добавление ключей БД отменено, качество модели не повышено')

                    print('Неразмечено объектов = ', unlabeled_rows)
                    print('Размечено объектов = ', len(df_X1_L_tmp))
                    #print('Меток поставлено = ', len(y_tmp))
           
            

    elif method == 'k_iters':
        quality_its =[] #временное, для графиков качества на итерации
        print('Метод  = ', method)
        print('Предельная вероятность = ', aggr_proba_th)

        #k=0       
        for i in range(1,co_train_k+1):

            print(40 * '/', '\n', 40 * '/')
            print('Итерация CoTrain = ', i)
            print(40 * '/', '\n', 40 * '/')
            
            if len(df_X1_U_tmp)>0:
                # Обучение
                clf1.fit(df_X1_L_tmp, y_tmp.ravel())
                clf2.fit(df_X2_L_tmp, y_tmp.ravel())

                model_qual_t0 = CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_data_test)
                # Оценка вероятностей классов
                proba_clf1 = clf1.predict_proba(df_X1_U_tmp[0:p_const])
                proba_clf2 = clf2.predict_proba(df_X2_U_tmp[0:p_const])
                unmkd_tmp = len(df_X1_U_tmp)

                aggr_proba = (proba_clf1 + proba_clf2)/2  

                pos_aggr_prob_ind = np.asarray(np.where(aggr_proba[ :, 1] > aggr_proba_th)).ravel()
                neg_aggr_prob_ind = np.asarray(np.where(aggr_proba[ :, 0] > aggr_proba_th)).ravel()

                randomize_p = np.arange(len(pos_aggr_prob_ind))
                randomize_n = np.arange(len(neg_aggr_prob_ind))

                np.random.shuffle(randomize_p)
                np.random.shuffle(randomize_n)
                pos_aggr_prob_ind = pos_aggr_prob_ind[randomize_p]
                neg_aggr_prob_ind = neg_aggr_prob_ind[randomize_n]

                pos_aggr_prob_ind = pos_aggr_prob_ind[ :p]
                neg_aggr_prob_ind= neg_aggr_prob_ind[:n]


                conf_examples = np.concatenate((pos_aggr_prob_ind,  neg_aggr_prob_ind))

                y_clss = []        # размеченный класс
                y_clss_proba = []  # вероятность принадлежности к 1 классу


                unmkd = unmkd_tmp - len(conf_examples)
                print('Разметка данных')
                print('Неразмеченных объектов= ', unmkd)

                # добавление переменных по меткам классов
                df_X1_L_tmp_ch = copy.copy(np.vstack([df_X1_L_tmp, df_X1_U_tmp[conf_examples]]))
                df_X2_L_tmp_ch = copy.copy(np.vstack([df_X2_L_tmp, df_X2_U_tmp[conf_examples]]))

                y_clss = np.asarray(y_clss)
                y_clss = y_clss.reshape(len(y_clss), 1)
                y_clss_proba = np.asarray(y_clss_proba)
                y_clss_proba = y_clss_proba.reshape(len(y_clss_proba), 1)

                y_clss_p = np.ones( shape =(len(pos_aggr_prob_ind),1), dtype = int)
                y_clss_n = np.zeros( shape =(len(neg_aggr_prob_ind),1), dtype = int)

                #итоговые массивы для добавления
                y_clss = np.vstack([y_clss_p, y_clss_n]) 
                y_clss_proba_norm = aggr_proba[conf_examples, 1]/(aggr_proba[conf_examples, 1]+aggr_proba[conf_examples, 0])  
                y_clss_proba = np.vstack((y_clss_proba, y_clss_proba_norm.reshape(len(y_clss_proba_norm),1)))

                #приведение предыдущих массивов к единому виду
                y_tmp = np.asarray(y_tmp)
                y_tmp = y_tmp.reshape(len(y_tmp), 1)

                y_tmp_prob = np.asarray(y_tmp_prob)
                y_tmp_prob = y_tmp_prob.reshape(len(y_tmp_prob), 1)

                # добавление меток класса
                y_tmp_ch = copy.copy(np.vstack([y_tmp, y_clss]))
                y_tmp_prob_ch = copy.copy(np.vstack([y_tmp_prob, y_clss_proba]))

                # корректировка неразмеченных массивов
                # backup
                df_X1_U_tmp_backup = df_X1_U_tmp.copy()
                df_X2_U_tmp_backup = df_X2_U_tmp.copy()

                df_X1_U_tmp_ch = np.delete(df_X1_U_tmp, conf_examples, axis=0)
                df_X2_U_tmp_ch = np.delete(df_X2_U_tmp, conf_examples, axis=0)

                #Автокоррекция подгонки модели
                clf1.fit(df_X1_L_tmp_ch, y_tmp_ch.ravel())
                clf2.fit(df_X2_L_tmp_ch, y_tmp_ch.ravel())

                model_qual_t1 = CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2, y_data_test)
                print('Качество до текущей разметки:', '{0:.5f}'.format(model_qual_t0))
                print('Качество после текущей разметки:', '{0:.5f}'.format(model_qual_t1))
                #print(len(df_X2_U_tmp_ch))
                #print(len(df_X1_U_tmp_backup))

                print('Согласованных объектов для разметки: ',len(conf_examples))
                print('Positive: ', pos_aggr_prob_ind)
                print('Negative: ', neg_aggr_prob_ind)
                print(conf_examples)
                if model_qual_t1 > model_qual_t0:
                    GREEN_LIGHT = 1
                    quality_its.append(model_qual_t1)
                    #k+=1
                    print('Качество повышено на ',model_qual_t1 - model_qual_t0, ', объекты размечаются')              
                    #можно объявить массивы основными данными для подгонки                
                    y_tmp = y_tmp_ch
                    y_tmp_prob = y_tmp_prob_ch

                    df_X1_L_tmp = df_X1_L_tmp_ch
                    df_X2_L_tmp = df_X2_L_tmp_ch

                    df_X1_U_tmp = df_X1_U_tmp_ch
                    df_X2_U_tmp = df_X2_U_tmp_ch

                else:
                    #k+=1 # убрать
                    GREEN_LIGHT = 0
                    quality_its.append(model_qual_t0)

                    df_X1_U_tmp = df_X1_U_tmp_backup
                    df_X2_U_tmp = df_X2_U_tmp_backup
                    objs_keys_U_tmp

                    randomize = np.arange(len(df_X1_U_tmp))
                    np.random.shuffle(randomize)

                    df_X1_U_tmp = df_X1_U_tmp[randomize]
                    df_X2_U_tmp = df_X2_U_tmp[randomize]
                    objs_keys_U_tmp = objs_keys_U_tmp[randomize]

                    print('Качество снижено, объекты не размечаются')


                unlabeled_rows = len(df_X1_U_tmp)


                if len(objs_keys) == 0:
                    print('objs_keys is None, Ключи для БД не заданы')
                else:
                    print('objs_keys is not None, Добавление ключей БД')
                    if GREEN_LIGHT == 1:
                        objs_keys_L_tmp = np.vstack([objs_keys_L_tmp, objs_keys_U_tmp[conf_examples]])
                        objs_keys_U_tmp = np.delete(objs_keys_U_tmp, conf_examples, axis=0)
                    else:
                        print('Добавление ключей БД отменено, качество модели не повышено')

                print('Неразмечено объектов = ', unlabeled_rows)
                print('Размечено объектов = ', len(df_X1_L_tmp))
                #print('Меток поставлено = ', len(y_tmp))
            
            else:
                print('Разметка завершена раньше заданного чсила итераций CoTrain')


        if len(df_X1_U_tmp) > 0:

            #Единая разметка оставшихся объектов
            proba_clf1 = clf1.predict_proba(df_X1_U_tmp)
            proba_clf2 = clf2.predict_proba(df_X2_U_tmp)
            aggr_proba = (proba_clf1 + proba_clf2)/2
            
            y_conv_1_proba = np.asarray([])
            y_conv_1_proba = y_conv_1_proba.reshape(len(y_conv_1_proba), 1)
            
            logic = aggr_proba[:,1]> aggr_proba[:,0] #True - класс 1, False - класс 0
            
            pos = np.where(logic==True) 
            neg = np.where(logic==False) 
            all_ind = np.concatenate((pos[0],  neg[0]))
            
            y_clss_p = np.ones( shape =(len(pos[0]),1), dtype = int)
            y_clss_n = np.zeros( shape =(len(neg[0]),1), dtype = int)
            
            y_conv_1 = np.vstack([y_clss_p, y_clss_n]) 
            y_conv_proba_norm = aggr_proba[all_ind, 1]/(aggr_proba[all_ind, 1]+ aggr_proba[all_ind, 0])
            y_conv_1_proba = np.vstack((y_conv_1_proba, y_conv_proba_norm.reshape(len(y_conv_proba_norm),1)))
            

            # добавление переменных по меткам классов
            df_X1_L_tmp = np.vstack([df_X1_L_tmp, df_X1_U_tmp[all_ind]])
            df_X2_L_tmp = np.vstack([df_X2_L_tmp, df_X2_U_tmp[all_ind]])
            
            y_clss = np.asarray(y_conv_1)
            y_clss = y_clss.reshape(len(y_clss), 1)
            
            y_clss_proba = np.asarray(y_conv_1_proba)
            y_clss_proba = y_clss_proba.reshape(len(y_clss_proba), 1)

            y_tmp = np.asarray(y_tmp)
            y_tmp = y_tmp.reshape(len(y_tmp), 1)
            
            y_tmp_prob = np.asarray(y_tmp_prob)
            y_tmp_prob = y_tmp_prob.reshape(len(y_tmp_prob), 1)

            y_tmp = np.vstack([y_tmp, y_clss])
            y_tmp_prob = np.vstack([y_tmp_prob, y_clss_proba])
            
            df_X1_U_tmp = np.delete(df_X1_U_tmp, all_ind, axis=0)
            df_X2_U_tmp = np.delete(df_X2_U_tmp, all_ind, axis=0)
            
            unlabeled_rows = len(df_X1_U_tmp)
            
            if len(objs_keys) == 0:
                print('objs_keys is None, Ключи для БД не заданы')
            else:
                print('objs_keys is not None, Добавление ключей БД')
                objs_keys_L_tmp = np.vstack([objs_keys_L_tmp, objs_keys_U_tmp[all_ind]])
                objs_keys_U_tmp = np.delete(objs_keys_U_tmp, all_ind, axis=0)

            print('Неразмечено объектов = ', unlabeled_rows)
            print('Размечено объектов = ', len(df_X1_L_tmp))
            #print('Меток поставлено = ', len(y_tmp))


    # если тестируем качество
    if dataframe_test is not None:
        
        y_test_pred =  CoTrain_Classifier_Test(dataframe_test, X1, X2, clf1, clf2)


    # если привязываем к ключам БД
    if len(objs_keys) != 0:
        result = pd.DataFrame({'Client class': y_tmp.ravel(),
                               'Switcher prob': y_tmp_prob.ravel(),
                               'Client ID': objs_keys_L_tmp.ravel()})
        #result = [y_tmp,y_tmp_prob]
        if dataframe_test is None:
            #вернуть предсказанное
            return result, quality_its
        # если тестируем качество и выводим весь предсказанный массив объектов
        if dataframe_test is not None:
            return result, y_test_pred, quality_its  # все размеченные данные #внешние данные для  тестирования

    else:
        result = y_tmp.ravel()
        if dataframe_test is None:
            return result, y_test_pred, quality_its
        if dataframe_test is None:
            return result,quality_its
